In [11]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
import couchdb
import json
import pickle
from time import sleep

In [2]:
# Set ES connection
es = Elasticsearch('http://10.10.100.48:32793')
es.ping()

True

In [4]:
include = ['taxon.scientificName']
body = {"query": {"match_all": {}}}
results = es.search(index='livro_vermelho_2013', doc_type='assessment', body=body, _source_include=include)
print "Foram recuperados %d assessments" % results['hits']['total']

Foram recuperados 4617 assessments


Recuperando todos os assessments do Livro Vermelho 2013 e os profiles relacionados

In [5]:
include = ['taxon.scientificName']
query = json.loads('{"query": {"match_all": {}}}')
assessments = scan(es, index='livro_vermelho_2013', doc_type='assessment', query=query, _source_include=include)

profiles_to_name = {}
assessments_to_name = {}
assessments_to_profile = {}
body = json.loads('''{
    "query": {"match": {
       "taxon.scientificName": {
           "query": "%s", 
           "type": "phrase"
    }
}}}''')

print "Verificando se os assessments possuem somente um profile"
ok = True
for item in assessments:
    body['query']['match']['taxon.scientificName']['query'] = item['_source']['taxon']['scientificName']
    profile = es.search(index='livro_vermelho_2013', doc_type='profile', body=body, _source_include=include)
    total = profile['hits']['total']
    if total > 1:
        print "%s: %d" % (item['_source']['taxon']['scientificName'], total)
        ok = False
    assessments_to_name[item['_id']] = item['_source']['taxon']['scientificName']
    profiles_to_name[profile['hits']['hits'][0]['_id']] = profile['hits']['hits'][0]['_source']['taxon']['scientificName']
    assessments_to_profile[item['_id']] = profile['hits']['hits'][0]['_id']

if ok == True:
    print "OK"
else:
    print "Verifique os assessments acima"
    
print "Foram verificados %d assessments" % len(assessments_to_profile)
pickle.dump( assessments_to_profile, open( "assessments_to_profile.pkl", "wb" ) )

Verificando se os assessments possuem somente um profile
OK
Foram verificados 4617 assessments


Conectando ao CouchDB e copiando datas de Assessment para Profile

In [21]:
couch = couchdb.Server('http://jb049/couchdb')
db = couch['livro_vermelho_2013']
update_body = json.loads('''{
    "doc": {
        "metadata": {
            "created": "%d",
            "modified": "%d"
        },
        "rev": "%s"
    }
}''')

docs = []
docs_profile = []
for id_assess, id_profile in assessments_to_profile.iteritems():
    doc_assess = db.get(id_assess)
    doc_profile = db.get(id_profile)
    doc_profile['metadata']['created'] = doc_assess['metadata']['created']
    doc_profile['metadata']['modified'] = doc_assess['metadata']['modified']
    docs.append([doc_profile])

Copying assessment 54bfbc97b1a28 to profile 54ff180da49c9
Copying assessment 54bfbc98189f5 to profile 54ff180dd0d75
Copying assessment 54bfbc81530a5 to profile 54ff18077a72d
Copying assessment 54bfbc732727e to profile 54ff1803c58b2
Copying assessment 54bfbc99b9c3d to profile 54ff180e331ce
Copying assessment 54bfbc9699a48 to profile 54ff180d1697e
Copying assessment 54bfbc8d5dcc0 to profile 54ff180a57bd8
Copying assessment 54bfbc810153d to profile 54ff18072672f
Copying assessment 54bfbc7eb98a5 to profile 54ff1806cf4f1
Copying assessment 54bfbc7ce9dbd to profile 54ff18067c493
Copying assessment 54bfbc9848082 to profile 54ff180dd213a
Copying assessment 54bfbc8707c05 to profile 54ff180919f32
Copying assessment 54bfbc8a0e940 to profile 54ff1809a1073
Copying assessment 54bfbc5c1ccd0 to profile 54ff17fd77061
Copying assessment 54bfbc8d771f3 to profile 54ff180ac80ed
Copying assessment 54bfbc7677ab5 to profile 54ff18048dd49
Copying assessment 54bfbc79705cd to profile 54ff1805448e4
Copying assess

ServerError: (413, '')

In [44]:
i = 0
doc_profile = docs[0]
while doc_profile and i < len(docs):
    try:
        response = db.update([doc_profile])
        update_body['doc']['metadata']['created'] = doc_profile['metadata']['created']
        update_body['doc']['metadata']['modified'] = doc_profile['metadata']['modified']
        update_body['doc']['rev'] = response[0][2]
        id_profile = response[0][1]
        result = es.update(index='livro_vermelho_2013', doc_type='profile', id=id_profile, body=update_body)
        i +=1
    except couchdb.ServerError:
        print "Server error for id %s" % doc_profile['_id']
        sleep(30)
        
    doc_profile = docs[i]

Server error for id %s 54ff180af1163
Server error for id %s 54ff180619854
Server error for id %s 54ff1803c5d98


IndexError: list index out of range

Verificando se docs estão iguais no CouchDB e no ES

In [46]:
for id_assess, id_profile in assessments_to_profile.iteritems():
    doc_profile = db.get(id_profile)
    doc_profile_es = es.get(index='livro_vermelho_2013', doc_type='profile', id=id_profile)
    doc_profile_es['_source']['_id'] = doc_profile_es['_source']['id']
    del doc_profile_es['_source']['id']
    doc_profile_es['_source']['_rev'] = doc_profile_es['_source']['rev']
    del doc_profile_es['_source']['rev']
    if doc_profile != doc_profile_es['_source']:
        print "Check document %s, profile in CouchDB is different than ES" % id_profile

In [47]:
db.commit()

{u'instance_start_time': u'1430147074930688', u'ok': True}